## Callin Switzer
15 November 2016
Update: 10 Feb 2017

- Read in frames from raw .avi video, and stack them all up to visualize pollen release over time
- This is used to make a figure and a supplemental video for manuscript

In [1]:
import cv2
import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk

# scikit image
import skimage
from skimage import io

import os
import pandas as pd
import time

/Users/callinswitzer/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
%qtconsole

In [3]:
import Tkinter as tk
import tkFileDialog

def file_choose():
    '''
    Opens tkfileDialog and returns full path name.
    It's the equivalent of file.choose in R.
    Make sure the following modeules are loaded
    import Tkinter as tk
    import tkFileDialog
    Also, won't work if %matplotlib qt4 has been used
    '''
    root = tk.Tk()
    root.update()
    file1 = tkFileDialog.askopenfilename(parent=root)
    root.destroy() # closes the window
    return file1

In [5]:
# get this video: 20160615_153850.avi
aa = '/Users/callinswitzer/Dropbox/ExperSummer2016/Kalmia/Manuscript/DatasetsSupplemental/20160615_153850.avi'

In [6]:
print aa
cap = cv2.VideoCapture(aa)
print 'numFrames = ' + str(cap.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT))

/Users/callinswitzer/Dropbox/ExperSummer2016/Kalmia/Manuscript/DatasetsSupplemental/20160615_153850.avi
numFrames = 1003.0


In [7]:
# read in frames and stack them up as a np array

# read in one frame
ret, frame = cap.read()

# convert to grayscale
frame = np.array(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
frame.shape


numFr = int(np.round(cap.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT)))

# preallocate array
imStack = np.ones(np.hstack([list(frame.shape), numFr])) * 0.0


In [8]:
cap = cv2.VideoCapture(aa)

for ii in range(numFr):
    ret, frame = cap.read()
    frame = 255 - np.array(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
    imStack[:,:,ii] = frame
    print ii

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [9]:
# get median value for each pixel over time
aa = np.median(imStack, axis = 2)

In [10]:
# subtract background
im2 = np.array([imStack[:,:,ii] - aa for ii in range(imStack.shape[2])])

In [16]:
# threshold
bb = np.array([im2[ii,:,:] > 20 for ii in range(imStack.shape[2])])

In [18]:
# show another bw image
ii = 500
plt.clf()
io.imshow(imStack[:,:,ii] > 60, cmap = 'gray')

In [19]:
# make a normalized image stack -- moving things will be black
imst2 = 1 - (imStack / np.max(imStack))

In [20]:
# make new video to combine two image stacks
AnnotStack = np.array([imst2[:, :, ii] * (im2[ii,:,:] < 20) for ii in range(imStack.shape[2])])

In [21]:
# save video
vidDir = '/Users/callinswitzer/Desktop/testStack2/'

if not os.path.isdir(vidDir):
    os.mkdir(vidDir)

os.chdir(vidDir)

In [22]:
# save images -- no tail
for ii in range(AnnotStack.shape[0]):
    cv2.imwrite(str(ii).zfill(4) + '.png', AnnotStack[ii, :,:] * 255.0)

In [23]:
# make images into a movie
stta = time.time()
!ffmpeg -start_number 0 -i %04d.png -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -c:v libx264 -pix_fmt yuv420p -y FlyingPollenOverlay.mp4
print time.time() - stta # ~ 1.4 seconds

ffmpeg version 3.1.4 Copyright (c) 2000-2016 the FFmpeg developers
  built with Apple LLVM version 8.0.0 (clang-800.0.38)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/3.1.4 --enable-shared --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-opencl --enable-libx264 --enable-libmp3lame --enable-libxvid --disable-lzma --enable-vda
  libavutil      55. 28.100 / 55. 28.100
  libavcodec     57. 48.101 / 57. 48.101
  libavformat    57. 41.100 / 57. 41.100
  libavdevice    57.  0.101 / 57.  0.101
  libavfilter     6. 47.100 /  6. 47.100
  libavresample   3.  0.  0 /  3.  0.  0
  libswscale      4.  1.100 /  4.  1.100
  libswresample   2.  1.100 /  2.  1.100
  libpostproc    54.  0.100 / 54.  0.100
Input #0, image2, from '%04d.png':
  Duration: 00:00:40.12, start: 0.000000, bitrate: N/A
    Stream #0:0: Video: png, gray(pc), 720x600, 25 fps, 25 tbr, 25 tbn, 25 tbc
[libx264 @ 0x7fe6f2803a00] using

In [24]:
#annotate with tail by stacking images
im3 = np.array([(im2[ii,:,:] < 20) + (im2[ii-1,:,:] < 20) for ii in range(1, imStack.shape[2])])

In [25]:
im3 = im2.copy()
im3[0, :,:] = 0

for ii in range(1, im2.shape[0]):
    im3[ii,:,: ] = (im3[ii,:,: ] > 10) + (im3[ii-1, :, :] > 0)

In [26]:
plt.clf()
io.imshow(im3[0, :,:] > 0)

In [27]:
plt.clf()
io.imshow(im3[500, :,:])

In [28]:
# combine stacks
AnnotStack2 = np.array([imst2[:, :, ii] * (im3[ii,:,:] < 1) for ii in range(imStack.shape[2])])

In [29]:
# save images
vidDir = '/Users/callinswitzer/Desktop/testStack3/'

if not os.path.isdir(vidDir):
    os.mkdir(vidDir)

os.chdir(vidDir)

# save images
for ii in range(AnnotStack.shape[0]):
    cv2.imwrite(str(ii).zfill(4) + '.png', AnnotStack2[ii, :,:] * 255.0)

In [30]:
# convert images to video
stta = time.time()
!ffmpeg -start_number 0 -i %04d.png -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -c:v libx264 -pix_fmt yuv420p -y FlyingPollenOverlay_tail.mp4
print time.time() - stta # ~ 1.4 seconds

ffmpeg version 3.1.4 Copyright (c) 2000-2016 the FFmpeg developers
  built with Apple LLVM version 8.0.0 (clang-800.0.38)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/3.1.4 --enable-shared --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-opencl --enable-libx264 --enable-libmp3lame --enable-libxvid --disable-lzma --enable-vda
  libavutil      55. 28.100 / 55. 28.100
  libavcodec     57. 48.101 / 57. 48.101
  libavformat    57. 41.100 / 57. 41.100
  libavdevice    57.  0.101 / 57.  0.101
  libavfilter     6. 47.100 /  6. 47.100
  libavresample   3.  0.  0 /  3.  0.  0
  libswscale      4.  1.100 /  4.  1.100
  libswresample   2.  1.100 /  2.  1.100
  libpostproc    54.  0.100 / 54.  0.100
Input #0, image2, from '%04d.png':
  Duration: 00:00:40.12, start: 0.000000, bitrate: N/A
    Stream #0:0: Video: png, gray(pc), 720x600, 25 fps, 25 tbr, 25 tbn, 25 tbc
[libx264 @ 0x7f9f0b009c00] using

In [1]:
# show installed packages and versions
!pip freeze 

alabaster==0.7.9
anaconda-clean==1.0
anaconda-client==1.5.1
anaconda-navigator==1.3.1
appnope==0.1.0
appscript==1.0.1
argcomplete==1.0.0
astroid==1.4.7
astropy==1.2.1
Babel==2.3.4
backports-abc==0.4
backports.shutil-get-terminal-size==1.0.0
backports.ssl-match-hostname==3.4.0.2
beautifulsoup4==4.5.1
bitarray==0.8.1
blaze==0.10.1
bokeh==0.12.2
boto==2.42.0
Bottleneck==1.1.0
cdecimal==2.3
cffi==1.7.0
chest==0.2.3
click==6.6
cloudpickle==0.2.1
clyent==1.2.2
colorama==0.3.7
conda==4.2.11
conda-build==2.0.2
configobj==5.0.6
configparser==3.5.0
contextlib2==0.5.3
cryptography==1.3.1
cycler==0.10.0
Cython==0.24.1
cytoolz==0.8.0
dask==0.11.0
datashape==0.5.2
decorator==4.0.10
dill==0.2.5
docutils==0.12
dynd==0.7.3.dev1
enum34==1.1.6
et-xmlfile==1.0.1
fastcache==1.0.2
filelock==2.0.6
Flask==0.11.1
Flask-Cors==2.1.2
funcsigs==1.0.2
functools32==3.2.3.post2
futures==3.0.5
gevent==1.1.2
greenlet==0.4.10
grin==1.2.1
h5py==2.6.0
HeapDict==1.0.0
html5lib==0.999
idna==2.1
imagesize==0.7.1
ipaddress==1

In [2]:
# print system info
import IPython
print IPython.sys_info()

{'commit_hash': u'5c9c918',
 'commit_source': 'installation',
 'default_encoding': 'UTF-8',
 'ipython_path': '/Users/callinswitzer/anaconda/lib/python2.7/site-packages/IPython',
 'ipython_version': '5.1.0',
 'os_name': 'posix',
 'platform': 'Darwin-15.6.0-x86_64-i386-64bit',
 'sys_executable': '/Users/callinswitzer/anaconda/bin/python',
 'sys_platform': 'darwin',
 'sys_version': '2.7.9 |Anaconda custom (x86_64)| (default, Dec 15 2014, 10:37:34) \n[GCC 4.2.1 (Apple Inc. build 5577)]'}
